In [68]:
import clingo

from starlingo.Atom import Atom
from starlingo.Literal import Literal, BasicLiteral
from starlingo.Program import from_string, Program, evaluate_forwards
from starlingo.Rule import NormalRule, Fact
from starlingo.Symbol import Function

In [69]:
simple_str = """

a :- b, d.
b :- d.
c :- d.
d.

"""
simple_p = from_string(simple_str)[0]
print(simple_p.custom_str(sep='\n'))

#program base.
a :- b, d.
b :- d.
c :- d.
d.


In [70]:
tuple(evaluate_forwards((simple_p,), report=True));

Answer 1: {
a
b
c
d
}
SAT 1


In [71]:
dual_rules = []
for rule in simple_p.rules:
    if rule.is_rule() and rule.is_normal_rule():
        head = -rule.head
        body = tuple(-literal for literal in rule.body)
        for literal in body:
            dual_rule = NormalRule(head, (literal,))
            dual_rules.append(dual_rule)
        if not body:
            dual_rule = Fact(head)
            dual_rules.append(dual_rule)
simple_d = Program(rules=dual_rules)
print(simple_d.custom_str(sep='\n'))

#program base.
not a :- not b.
not a :- not d.
not b :- not d.
not c :- not d.
not d.


In [72]:
tuple(evaluate_forwards((simple_d,), report=True));

Answer 1: {

}
SAT 1


<block>:1:20-21: info: atom does not occur in any rule head:
  a

<block>:1:29-30: info: atom does not occur in any rule head:
  b

<block>:1:36-37: info: atom does not occur in any rule head:
  a

<block>:1:45-46: info: atom does not occur in any rule head:
  d

<block>:1:52-53: info: atom does not occur in any rule head:
  b

<block>:1:61-62: info: atom does not occur in any rule head:
  d

<block>:1:68-69: info: atom does not occur in any rule head:
  c

<block>:1:77-78: info: atom does not occur in any rule head:
  d

<block>:1:84-85: info: atom does not occur in any rule head:
  d



In [73]:
tuple(evaluate_forwards((simple_p,simple_d,), report=True));

UNSAT

In [74]:
simple_neg_str = """

a :- not b.
b :- not a.

"""
simple_neg_p = from_string(simple_neg_str)[0]
print(simple_neg_p.custom_str(sep='\n'))

#program base.
a :- not b.
b :- not a.


In [75]:
tuple(evaluate_forwards((simple_neg_p,), report=True));

Answer 1: {
b
}
Answer 2: {
a
}
SAT 2


In [76]:
dual_rules = []
for rule in simple_neg_p.rules:
    if rule.is_rule() and rule.is_normal_rule():
        head = -rule.head
        body = tuple(-literal for literal in rule.body)
        for literal in body:
            dual_rule = NormalRule(head, (literal,))
            dual_rules.append(dual_rule)
        if not body:
            dual_rule = Fact(head)
            dual_rules.append(dual_rule)
simple_neg_d = Program(rules=dual_rules)
print(simple_neg_d.custom_str(sep='\n'))

#program base.
not a :- b.
not b :- a.


In [77]:
tuple(evaluate_forwards((simple_neg_d,), report=True));

Answer 1: {

}
SAT 1


<block>:1:20-21: info: atom does not occur in any rule head:
  a

<block>:1:25-26: info: atom does not occur in any rule head:
  b

<block>:1:32-33: info: atom does not occur in any rule head:
  b

<block>:1:37-38: info: atom does not occur in any rule head:
  a



In [78]:
tuple(evaluate_forwards((simple_neg_p, simple_neg_d,), report=True));

Answer 1: {
b
}
Answer 2: {
a
}
SAT 2


In [79]:
tweety_str = """

fly(X) :- bird(X), not -fly(X).
bird(X) :- penguin(X).
-fly(X) :- penguin(X).

bird(tim).
penguin(tweety).

"""
tweety_p = from_string(tweety_str)[0]
print(tweety_p.custom_str(sep='\n'))

#program base.
fly(X) :- bird(X), not -fly(X).
bird(X) :- penguin(X).
-fly(X) :- penguin(X).
bird(tim).
penguin(tweety).


In [80]:
tuple(evaluate_forwards((tweety_p,), report=True));

Answer 1: {
bird(tim)
bird(tweety)
fly(tim)
penguin(tweety)
-fly(tweety)
}
SAT 1


In [81]:
dual_rules = []
for rule in tweety_p.rules:
    if rule.is_rule() and rule.is_normal_rule():
        head = -rule.head
        body = tuple(-literal for literal in rule.body)
        phony_head = BasicLiteral(sign=rule.head.sign, atom=rule.head.atom.name_prepend('__'))
        phony_body = tuple(BasicLiteral(sign=literal.sign, atom=literal.atom.name_prepend('__')) for literal in rule.body)
        dual_rules.append(NormalRule(phony_head, phony_body))
        if rule.is_ground():
            for literal in body:
                rule = NormalRule(head, (literal,))
                dual_rules.append(rule)
            if not body:
                dual_rules.append(Fact(head))
        else:
            for literal in body:
                if not head.is_ground() and literal.is_neg():
                    head_variables = head.get_variables()
                    body_variables = literal.get_variables()
                    foralls = []
                    for head_variable in head_variables:
                        forall = BasicLiteral(Atom(Function('forall', (head_variable, -literal))))
                        foralls.append(forall)
                        phony_literal = BasicLiteral(literal.atom.name_prepend('__'))

                        forall_rule = NormalRule(forall, (phony_literal,))
                        dual_rules.append(forall_rule)
                    dual_rule = NormalRule(head, (literal, *foralls))

                else:
                    dual_rule = NormalRule(head, (literal,))
                dual_rules.append(dual_rule)
tweety_d = Program(rules=dual_rules)
print(tweety_d.custom_str(sep='\n'))

#program base.
__fly(X) :- __bird(X), not -__fly(X).
forall(X,bird(X)) :- __bird(X).
not fly(X) :- not bird(X), forall(X,bird(X)).
not fly(X) :- -fly(X).
__bird(X) :- __penguin(X).
forall(X,penguin(X)) :- __penguin(X).
not bird(X) :- not penguin(X), forall(X,penguin(X)).
-__fly(X) :- __penguin(X).
forall(X,penguin(X)) :- __penguin(X).
not -fly(X) :- not penguin(X), forall(X,penguin(X)).
__bird(tim) :- .
not bird(tim).
__penguin(tweety) :- .
not penguin(tweety).


In [82]:
tuple(evaluate_forwards((tweety_d,), report=True));

Answer 1: {
__bird(tim)
__bird(tweety)
__fly(tim)
__penguin(tweety)
forall(tim,bird(tim))
forall(tweety,bird(tweety))
forall(tweety,penguin(tweety))
-__fly(tweety)
}
SAT 1


<block>:1:90-96: info: atom does not occur in any rule head:
  fly(X)

<block>:1:104-111: info: atom does not occur in any rule head:
  bird(X)

<block>:1:136-142: info: atom does not occur in any rule head:
  fly(X)

<block>:1:146-153: info: atom does not occur in any rule head:
  (-fly(X))

<block>:1:224-231: info: atom does not occur in any rule head:
  bird(X)

<block>:1:239-249: info: atom does not occur in any rule head:
  penguin(X)

<block>:1:342-349: info: atom does not occur in any rule head:
  (-fly(X))

<block>:1:357-367: info: atom does not occur in any rule head:
  penguin(X)

<block>:1:412-421: info: atom does not occur in any rule head:
  bird(tim)

<block>:1:450-465: info: atom does not occur in any rule head:
  penguin(tweety)



In [83]:
tuple(evaluate_forwards((tweety_p,tweety_d,), report=True));

UNSAT